Importing nessesarry libraries

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm #progress bar
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.utils import save_image

from IPython.display import HTML

In [2]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setting train dataset location
train_dir = '/content/drive/MyDrive/Dataset/Celeb/celebData/train'

In [ ]:
# Defining the hyper-parameters 

# Batch Size for the dataset
BATCH_SIZE = 128

# setting image sizes for our dataset 
# normally 178*218 however we will reduce it to 64 for the sake of trainning
IMG_SIZE = 64

# setting the number of workers
WORKERS = 2

# number of channels for images in our case is 3 colors (RGB)
CHANNELS_NUMBER = 3

# setting the size of the feature maps for generator
gen_feature = 64

# setting the size of the feature maps for discriminator
dis_feature = 64

# setting the number of epochs
EPOCH_NUMBER = 10

# setting the learning rate 
lr = 0.0002

Resize: This resizes the image to a square of size IMG_SIZE while maintaining the aspect ratio of the original image.

CenterCrop: This crops the center of the image to a square of size IMG_SIZE.

ToTensor: This converts the image to a PyTorch tensor.

Normalize: This normalizes the tensor image with mean and standard deviation (0.5, 0.5, 0.5) for each color channel. The values are normalized to be between -1 and 1.


In [ ]:
transform=transforms.Compose([
                               transforms.Resize(IMG_SIZE),
                               transforms.CenterCrop(IMG_SIZE),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

The train_dataset variable is defined using the ImageFolder class from the torchvision.datasets module. This class assumes that the data is organized in a directory structure such that each subdirectory corresponds to a different class, and the images within each subdirectory belong to that class. The root argument specifies the path to the directory containing the training data, and the transform argument specifies the image transformations to be applied to the data. Here we actually do not need to have any labels since it is unsupervised learning.

The train_loader variable is defined using the DataLoader class from the torch.utils.data module. This class provides an iterable over the dataset, allowing the model to access the training data in batches during training. The batch_size argument specifies the number of samples in each batch, shuffle argument specifies whether to shuffle the data between epochs, pin_memory argument allows for faster transfer of data to the GPU, and drop_last argument drops the last batch of data if it's smaller than the batch size.

Overall, this code sets up a DataLoader for the training data with batch size BATCH_SIZE and image transformations transform. This DataLoader can be used to train a deep learning model on the training data.

In [ ]:
train_dataset = dset.ImageFolder(root=train_dir,
                           transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
    shuffle=True, pin_memory=True,drop_last=True)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

This code generates a grid of sample images from the training data using the make_grid function provided by the torchvision.utils module. The real_batch variable is obtained by calling next(iter(train_loader)), which returns the next batch of data from the train_loader object.

The make_grid function takes a tensor of images and arranges them in a grid with a specified padding. In this case, it takes the first 64 images from real_batch and applies a padding of 2 pixels between each image. The normalize=True argument normalizes the image data to the range [0, 1]. Finally, the cpu() method is called to move the tensor to the CPU if it was previously on the GPU.

The resulting grid of images is displayed using matplotlib's imshow function. The figure function creates a new figure with a size of 8x8 inches, and the axis function is called with "off" to remove the axis labels. The title function sets the title of the plot to "Training Images".

Overall, this code is useful for visualizing a sample of the training data to get an idea of what kind of images the model will be trained on.

In [ ]:
real_batch = next(iter(train_loader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

This is a PyTorch implementation of a Generative Adversarial Network (GAN) generator model. The generator takes a 100-dimensional noise vector as input and outputs a 3-channel image of size 64x64.

The model architecture consists of a series of transposed convolutional layers, with each layer gradually increasing the spatial size of the output and reducing the number of channels. Batch normalization is applied to each layer to improve training stability. The final layer uses the hyperbolic tangent activation function to scale the pixel values of the output to the range [-1, 1], which is common for image data

In [ ]:
class Generator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.main = nn.Sequential(

            nn.ConvTranspose2d(100, 1024,kernel_size= 4, stride= 2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),

            nn.ConvTranspose2d(1024, 512, kernel_size= 4, stride=2,padding= 1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, kernel_size= 4, stride=2,padding= 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, kernel_size= 4, stride=2,padding= 1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, kernel_size= 4, stride=2,padding= 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, kernel_size= 4, stride=2,padding= 1, bias=False),
            nn.Tanh()
            )

    def forward(self, input):
        return self.main(input)

This code defines a Discriminator class that inherits from the nn.Module class of PyTorch. It contains a main sequential model that consists of several layers that process an input image and output a scalar value indicating whether the input is real or fake.

The layers in the main model are as follows:

-Conv2d layer with 3 input channels and 64 output channels, using a kernel size of 4, stride of 2, and padding of 1. It applies a leaky ReLU activation function with a negative slope of 0.2 to the output.

-Conv2d layer with 64 input channels and 128 output channels, using a kernel size of 4, stride of 2, and padding of 1. It applies batch normalization and a leaky ReLU activation function with a negative slope of 0.2 to the output.

-Conv2d layer with 128 input channels and 256 output channels, using a kernel size of 4, stride of 2, and padding of 1. It applies batch normalization and a leaky ReLU activation function with a negative slope of 0.2 to the output.

-Conv2d layer with 256 input channels and 512 output channels, using a kernel size of 4, stride of 2, and padding of 1. It applies batch normalization and a leaky ReLU activation function with a negative slope of 0.2 to the output.

-Conv2d layer with 512 input channels and 1 output channel, using a kernel size of 4, stride of 1, and padding of 0. It applies a sigmoid activation function to the output, which is the final prediction of the discriminator.

-The input to the Discriminator is expected to be a 3-channel image with a size of 64x64 pixels.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(3, 64,  kernel_size= 4, stride=2,padding= 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(64, 64 * 2,  kernel_size= 4, stride=2,padding= 1, bias=True),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, kernel_size= 4, stride=2,padding= 1, bias=True),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8,  kernel_size= 4, stride=2,padding= 1, bias=True),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(64 * 8, 1,  kernel_size= 4, stride=1,padding= 0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create an instance of generator 
gen_net = Generator().to(device)

"""DataParallel is a PyTorch module that allows the parallelization of model training on multiple GPUs.
It divides the data into smaller batches and distributes the computation across multiple GPUs.
By doing this, it can significantly speed up the training process and improve model performance.

The second argument of the DataParallel constructor is a list of device IDs that specify which GPUs to use. 
In this case, which means the model will be parallelized on GPU 0.
In summary, this line of code sets up a DataParallel model that parallelizes the training of gen_net on GPU 0 if the device type is CUDA."""
if (device.type == 'cuda'):
    gen_net = nn.DataParallel(gen_net, [0])

# Printing the Generator model
print(gen_net)

In [ ]:
# Create the Discriminator
dis_net = Discriminator().to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda'):
    dis_net = nn.DataParallel(dis_net, [0])

# Print the model
print(dis_net)

The loss function is defined as Binary Cross-Entropy (BCE) loss, which is commonly used for binary classification problems, such as distinguishing real images from generated ones in GANs.

The variable beta1 is a hyperparameter used in the Adam optimizer, which controls the decay rate of the first moment estimate of the gradient.

Two Adam optimizers are created, one for the discriminator (dis_optimizer) and one for the generator (gen_optimizer). The parameters of each network are passed to their respective optimizer, along with the learning rate and beta1 value.

In [ ]:
# define loss function
loss_fn = nn.BCELoss()

beta1 = 0.5
# Setup Adam optimizers for both G and D
dis_optimizer = optim.Adam(dis_net.parameters(), lr=lr, betas=(beta1, 0.999))
gen_optimizer = optim.Adam(gen_net.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
fake_list = []
gen_losses = []
dis_losses =[]
# Train the GAN
for epoch in tqdm(range(EPOCH_NUMBER)):
    for i, (X_batch, _) in enumerate(train_loader):
        # Move the images and labels to the device
        X_batch = X_batch.to(device)

        # Train the discriminator on real images
        dis_optimizer.zero_grad()
        # check the torch img dimensions and set labels accordingly
        real_labels = torch.ones(128, 1, 1, 1).to(device)
        fake_labels = torch.zeros(128, 1, 1, 1).to(device)

        real_outputs = dis_net(X_batch)
        real_loss = loss_fn(real_outputs, real_labels)
        real_loss.backward()
        dis_optimizer.step()

        # Train the discriminator on fake images
        noise = torch.randn(BATCH_SIZE, 100, 1, 1, device=device)
        fake_images = gen_net(noise)
        fake_outputs = dis_net(fake_images.detach())
        fake_loss = loss_fn(fake_outputs, fake_labels)
        fake_loss.backward()
        dis_optimizer.step()

        # save the loss for discriminator
        dis_losses.append((real_loss.item() + fake_loss.item()))

        # Train the generator
        gen_optimizer.zero_grad()
        noise = torch.randn(BATCH_SIZE, 100, 1, 1, device=device)
        fake_images = gen_net(noise)
        fake_outputs = dis_net(fake_images)
        gen_loss = loss_fn(fake_outputs, real_labels)
        gen_losses.append(gen_loss)
        gen_loss.backward()
        gen_optimizer.step()
   
    save_image(fake_images, "/content/drive/MyDrive/Dataset/Fake_Images_new/{0:0=6d}.png" .format(epoch+1), nrow=5, padding=2,normalize=True)
    x_fake = fake_images.detach().cpu().numpy()

    for k in range(EPOCH_NUMBER):
        plt.subplot(2, 5, k+1)
        plt.imshow(x_fake[k].reshape(64,64,3))
       #  * 255).astype(np.uint8))
        plt.xticks([])
        plt.yticks([])

    plt.tight_layout()
    plt.show()
    
    print("Epoch: %d, D Loss: %.4f, G Loss: %.4f" % (epoch+1, real_loss.item() + fake_loss.item(), gen_loss.item()))

In [ ]:
plt.imshow(np.transpose(vutils.make_grid(fake_images[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
ge = torch.tensor(gen_losses).long()
de = torch.tensor(dis_losses).long()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training "+ str(EPOCH_NUMBER))
plt.plot(ge,label="Generator")
plt.plot(de,label="Discriminator")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('Losses During Training with '+str(EPOCH_NUMBER)+'epochs')
plt.show()

In [ ]:
torch.save(dis_net.state_dict(), "/content/drive/MyDrive/Colab Notebooks/NN PROJECT/Weights/discnewepoch.pt")
torch.save(gen_net.state_dict(), "/content/drive/MyDrive/Colab Notebooks/NN PROJECT/Weights/gennewepoch.pt")

In [ ]:
torch.save(gen_net, '/content/drive/MyDrive/Colab Notebooks/NN PROJECT/models/base_gen_new.pt')
torch.save(dis_net, '/content/drive/MyDrive/Colab Notebooks/NN PROJECT/models/base_dis_new.pt')